In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import fastparquet
import sys
import re
pd.set_option('display.max_columns', None)

In [3]:
class exc(Exception):
    def __init__(self, i):
        self.i = i

    def __str__(self):
        return f'自定義例外:{self.i}'

In [4]:
def get_html(url, headers):
    num_retry = 0  # 設定初始重試次數
    max_retry = 3  # 最大重試次數3次
    try:
        while num_retry <= max_retry:
            r = requests.get(url, headers=headers)
            if r.status_code == requests.codes.ok:  # 是否成功請求網頁
                print('request success: %s' % url)
                return r
            else:
                time.sleep(3)
                if num_retry == max_retry:  # 當重試三次後丟出錯誤
                    raise exc(url)
                print('retrying request')
                num_retry += 1
    except exc:
        print(exc)
        # 超過請求次數印出 http request eror 並將錯誤丟出function外
        print('http request error: %s' % url)
        raise
    except:
        # 出現意料之外的錯誤時印出錯誤並將錯誤丟出function外
        print('Unexcepted error: %s' % url, sys.exc_info())
        raise

In [5]:
def parse_html(r):
    soup = BeautifulSoup(r.text, 'lxml')
    if soup != None:
        print('parsing success')  # 印出成功解析網頁名
        return soup
    else:
        raise exc('parse_error')  # 當beautifulsoup解析失敗為Nonetype時，將錯誤丟出function外

In [56]:
url = 'https://www.sum.com.tw/carsearchlist.php?v=3&ctp_prt=&price1=0&price2=220&yearrange=&cararea=&carregion=&cartype=&carcolor=&brand=&model=&year1=&year2=&doors=&q=&show_type=&carrecommend=&comp_goldstore=&discount=&cc1=&cc2=&protection=&carvr=&page=&sort=&sortpage1=&gclid=&priceRange1=&priceRange2=&wd=&_de=0.5690229095899482&page=1'

headers={
    'Host':'www.sum.com.tw',
'Connection':'keep-alive',
'Accept':'text/plain, */*; q=0.01',
'X-Requested-With':'XMLHttpRequest',
'sec-ch-ua-mobile':'?0',
'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36',

'Accept-Encoding':'gzip, deflate, br',
'Accept-Language':'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
'Cookie':'PHPSESSID=qn203skntjuv8bkevorgrhg5g7; _gcl_au=1.1.1197458183.1624031200; _ga=GA1.3.849228594.1624031200; __auc=38660c2a17a1fcd839cb333ad40; _hjTLDTest=1; _hjid=13961593-1974-490c-8e9b-f0726e39cde0; _fbp=fb.2.1624031200855.1701953032; _gid=GA1.3.966926215.1624256678; _TUCI_T=sessionNumber+18799&pageView+18799; Tagtoo_pta=pta_03+_&lta+_tm^3Ad1056_; __asc=c83520af17a2d7bcacaaa8c8958; _TUCS=1; _TUCI=sessionNumber+2498&ECId+155&hostname+www.sum.com.tw&pageView+7493',
}
r=requests.get(url,headers=headers)

In [57]:
soup=BeautifulSoup(r.text,'lxml')
seo_list=soup.find_all('li',class_='seo_list')
list1=[]
df1=pd.DataFrame(columns=['id','car_brand','car_model','car_address','car_title','dealer_name'])

for li in seo_list:
    idnum=li.get('data-seo_name')
    
    car_info=li.find('h3').a.text.split('｜')
    
    car_brand=car_info[0].split(' ',1)[0]
    
    car_model=car_info[0].split(' ',1)[1]
    
    car_address=li.find('ul').find_next_sibling().find('li').text
    
    car_title=li.find('div',class_='carCondition').h3.text.strip()
    
    car_dealer=li.find('ul').find_next_sibling().find('li').find_next_sibling().text
    
    df1= df1.append({'id':idnum,'car_brand':car_brand,'car_model':car_model,
                     'car_address':car_address,'car_title':car_title,'dealer_name':car_dealer}, ignore_index=True)

# df1.to_parquet('sum_car.parquet',engine='fastparquet')
# df1.to_csv('sum_car.csv', encoding = 'utf-8-sig')

In [58]:
df1

,id,car_brand,car_model,car_address,car_title,dealer_name
0,576000,TOYOTA,ALTIS,臺中市,【全台最便宜不用再看了】少跑原版件認證車 正17年 小改款11.5代 美車!賞車下訂送大螢幕...,永貿汽車
1,575887,TOYOTA,INNOVA,屏東縣,【2.0客貨兩用★全原廠保養】省油耐操✔一手車✔ABS✔全原鈑件✔可全額貸✔,正大汽車
2,575862,HONDA,HR-V,臺南市,可網購!S旗艦版 一手只跑4.5萬 LED頭燈 六安 原廠影音 快撥 可全貸 不用殺價就很便宜!,金台南汽車(大同一店)
3,575825,MITSUBISHI,COLT PLUS,屏東縣,【PLUS儀錶板★僅跑6萬5】盲點偵測✔IKEY✔藍芽✔胎壓偵測✔倒車雷達✔ISOFIX兒童安全椅,正大汽車
4,575804,TOYOTA,VIOS,臺南市,可網購!只跑1.7萬 17年式改版 手自排 新引擎 ABS 雙安 代步好選擇 可全貸 不用殺...,金台南汽車(大同一店)
5,575723,LEXUS,IS,臺南市,IS200T 僅跑3萬6 一手車 原鈑件 原廠保養 里程保證,台新汽車
6,575662,MITSUBISHI,OUTLANDER,臺中市,【全台最便宜不用再看了】僅跑5萬原版件認證車 正17年 四輪傳動 前為公家機關主管座車 大螢...,永貿汽車
7,575654,LUXGEN,S5 TURBO ECO HYPER,臺中市,僅跑4.1萬公里，小改款，新車頭，ECO版，更省油、實惠，一手車，低里程，渦輪增壓，定速、循跡防滑,億昇汽車
8,575640,HONDA,CR-V,臺南市,可網購!18年式 S版 大包 ACC 電動尾門 電動椅 全景天窗 車道偏移 少跑里程保證 全...,金台南汽車(大同一店)
9,575624,HONDA,HR-V,臺南市,可網購!閃蝶藍 S旗艦版 一手車 原廠影音 LED 八安 全車原鈑件 只跑長途 可全貸 不...,金台南汽車(大同一店)


In [9]:
# np = df1['id'][0]
url_in = 'https://www.sum.com.tw/carinfo-576054.php'
headers={
    'Host':'www.sum.com.tw',
'Connection':'keep-alive',
'Accept':'text/plain, */*; q=0.01',
'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36',
'Cookie':'PHPSESSID=qn203skntjuv8bkevorgrhg5g7; _gcl_au=1.1.1197458183.1624031200; _ga=GA1.3.849228594.1624031200; __auc=38660c2a17a1fcd839cb333ad40; _hjTLDTest=1; _hjid=13961593-1974-490c-8e9b-f0726e39cde0; _fbp=fb.2.1624031200855.1701953032; _gid=GA1.3.966926215.1624256678; _TUCI_T=sessionNumber+18799&pageView+18799; Tagtoo_pta=pta_03+_&lta+_tm^3Ad1056_; __asc=c83520af17a2d7bcacaaa8c8958; _TUCS=1; _TUCI=sessionNumber+2498&ECId+155&hostname+www.sum.com.tw&pageView+7493',
}

df2=pd.DataFrame(columns=['id','car_mileage','car_year','car_price','car_cylinderVolumn','car_color','car_gearType','car_dirveMode','car_fuel','car_door','car_seat','dealer_address'])

r1=requests.get(url_in,headers=headers)

In [17]:
soup1=BeautifulSoup(r1.text,'lxml')

carDetail=soup1.find('div',class_='buyCarDetailTitleTextContent')

carPrice=soup1.find('div',class_='buyCarDetailTitleTextPrice')

dfver = verified_crawl(soup1)

car_price=carPrice.find('b').text

info=carDetail.div.find_next_siblings()
info_list=[]
try:
    for sp in info:
        print(sp)
        info_list.append(sp.span.text.strip())
except AttributeError:
    pass
except:
    print('Unexpect error')
    pass
photo_url = soup1.find('div',class_='buyCarDetailTitleImg').text

dealer_info = soup1.find('div', class_='buyCarDetailContentDealerContact').find_all('p')

dealer_address = dealer_address_crawl(soup1)
        
dfequ = equipment_crawl(soup1)

car_year=info_list[0]

car_cylinderVolumn=info_list[1]

car_color=info_list[2]

car_gearType=info_list[3]

car_driveMode=info_list[4]

car_fuel=info_list[5]

car_mileage=info_list[6]

car_door=info_list[7]

car_seat=info_list[8]




df2 = pd.DataFrame(columns=['id','car_mileage','car_year','car_price','car_cylinderVolumn','car_color','car_gearType',
                            'car_dirveMode','car_fuel','car_door','car_seat','dealer_address'])

df2 = df2.append({'id':574860,'car_mileage':car_mileage,'car_year':car_year,'car_price':car_price,
                 'car_cylinderVolumn':car_cylinderVolumn,'car_color':car_color,'car_gearType':car_gearType,
                 'car_dirveMode':car_driveMode,'car_fuel':car_fuel,'car_door':car_door,'car_seat':car_seat,'dealer_address':dealer_address}, ignore_index=True)
df_temp = pd.concat([df2,dfequ,dfver], axis=1)
df_temp

<div>年　份　：<span>2016</span></div>
<div>排氣量　：<span>2400</span></div>
<div>顏　色　：<span>白色系</span></div>
<div>變速系統：<span>自排</span></div>
<div>傳動系統：<span>4WD</span></div>
<div>引擎燃料：<span>汽油車</span></div>
<div>里程數　：<span>91424</span></div>
<div>車門數　：<span>5門</span></div>
<div>乘客數　：<span>5</span></div>
<div class="lineScan"><img class="lineIcon" src="images/lineScan_icon.svg"/><div class="qrCodebox"><img class="qrCode" srcset="https://www.sum.com.tw/comp_pics/100x100/-lineqrcode-202096122515574.jpg"/> </div> </div>


,id,car_mileage,car_year,car_price,car_cylinderVolumn,car_color,car_gearType,car_dirveMode,car_fuel,car_door,car_seat,dealer_address,外觀配備,鋁圈,天窗,全景天窗,電動尾門,電動滑門,防眩後視鏡,電動後視鏡,後視鏡方向指示燈,車頂行李架,空力套件,燈光配備,霧燈,前霧燈,HID頭燈,LED頭燈,LED日行燈,LED尾燈,主動轉向頭燈,光感應式頭燈,頭燈水平調整,迎賓踏板燈,內裝配備,皮椅,賽車椅,第三排座椅,電動窗,電動座椅,電動腰靠,後座分離傾倒,衛星導航,多功能行車資訊顯示器,液晶螢幕,抬頭顯示器,方向盤換檔控制,方向盤音響控制,換檔撥片,恆溫空調,左右獨立空調,後座出風口,車內粉塵過濾,後遮陽簾,其 他,4WD,備胎,一手車,渦輪增壓,晶片系統,核桃木飾板,氣壓懸吊,駕駛輔助,Keyless/免鑰匙系統,Push Start/按鈕式發車,感應式雨刷,定速系統,倒車雷達,車前距雷達,倒車顯影,PVM/環景系統,APA/自動停車系統,BSM/盲點偵測系統,StopStart/怠速熄火系統,HSA/斜坡起步輔助,LDWS/車道偏離警示系統,ACC/主動式巡航系統,安全輔助,中控鎖,防盜系統,安全氣囊,ABS/防鎖死煞車系統,EBD/煞車力道分配系統,BAS/煞車力道輔助系統,AEB/自動緊急煞車系統,TCS/循跡防滑系統,ESP/車身動態穩定系統,ISOFIX/兒童安全椅固定,兒童安全鎖,TPMS/胎壓監控,影音配備,CD,CD筒,VCD,DVD,USB插槽,中控螢幕,數位電視,藍牙通訊,音響主機,外部音源接入,三大保證,非計程車,四大保固,里程保證,YES認證
0,574860,91424,2016,48.8萬,2400,白色系,自排,4WD,汽油車,5門,5,屏東縣屏東市和生路二段220號(正大汽車創始店),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1


In [8]:
def equipment_crawl(soup):
    
    list1=[]
    
    equ1 = soup1.find(class_='buyCarDetailContentSpec').find_all('li')
    
    for x in equ1:
        list1.append(x.text.strip())

    list2=list1[:-6]

    arr=np.array(list2)

    dfequ = pd.DataFrame(np.zeros(shape=(1,len(list2)),dtype=int),columns=arr)
    for y in equ1[:-6]:
        try:
            e = y.text.strip()
            class_type = None
            class_type = y.get('class')[0]
            if class_type == 'specChecked':
                dfequ[e] = 1
        except IndexError:
                continue
    
    return dfequ

In [7]:
def dealer_address_crawl(soup):
    try:
        dealer_info = soup1.find('div', class_='buyCarDetailContentDealerContact').find_all('p')
        dealer_address = None
        for p in dealer_info:
            address = re.search('地址',p.text)
            if address:
                dealer_address = p.text.split('：')[1].strip()
                return dealer_address
    except:
        print('dealer_address parse error')
        return None
    

In [6]:
def verified_crawl(soup):
    carVerified = soup1.find('div', class_='buyCarDetailTitleTextBadge').find_all('li')
    
    dfver = pd.DataFrame(np.zeros(shape=(1,5),dtype=int),columns=['三大保證', '非計程車', '四大保固', '里程保證', 'YES認證'])
    
    for x in carVerified:
        v = x.text.strip()
        class_type = None
        
        try:
            class_type = x.get('class')[1]
            if class_type == 'carConditionActive':
                dfver[v] = 1
        except IndexError:
            continue
    
    return dfver